# Loop

In [ ]:
import Pipeline
import pandas as pd
import time
import os
import numpy as np

In [ ]:
# Define video and metadata paths
data_path = "insert_your_data_path_here"  # Replace with your actual data path
metadata_path = data_path + "/metadata" # Path to metadata files
video_path = data_path + "/video" # Path to video files

# Collect Video IDs
video_ids = os.listdir(video_path)
video_ids = [id.split(".")[0] for id in video_ids]

# Create several empty lists for counts
time_VID = []
time_IC = []
time_IS = []
time_OD = []
time_MC = []
time_SR = []
time_NR = []
time_TR = []
time_amount = []
time_unique = []

In [ ]:
def loop(video_id):
    
    model_start = time.time()
    
    # Extract metadata
    metadata = Pipeline.extract_metadata(metadata_path,video_id,evaluation=True)
    
    # Extract Frames
    frames, frame_amount = Pipeline.extract_frames(video_path,video_id,metadata,evaluation=True)
    
    # Duplicate Removal
    frames, unique_amount = Pipeline.drop_duplicates(frames,mse_threshold=0)
    
    # IC (Image Captioning)
    frames, IC_time = Pipeline.IC(frames)
    
    # IS (Image Segmentation)
    frames, IS_time = Pipeline.IS(frames)
    
    # OD (Object Detection)
    frames, OD_time = Pipeline.OD(frames)
    
    # MC (Multilabel Classification)
    frames, MC_time = Pipeline.MC(frames)
    
    # SR (Scene Recognition)
    frames, SR_time = Pipeline.SR(frames)
    
    # NR (Night Recognition)
    frames, NR_time = Pipeline.NR(frames)
    
    # TR (Text Recognition)
    frames, TR_time = Pipeline.TR(frames)
    
    # Summarization of Predictions
    summarization = Pipeline.summarize_predictions(frames,metadata)
    # Summarization saves results as csv: metadata_dataframe.to_csv("Summarizations.csv",index=False)
    
    # Append all video-related information to lists
    time_VID.append(video_id)
    time_IC.append(IC_time)
    time_IS.append(IS_time)
    time_OD.append(OD_time)
    time_MC.append(MC_time)
    time_SR.append(SR_time)
    time_NR.append(NR_time)
    time_TR.append(TR_time)
    time_amount.append(frame_amount)
    time_unique.append(unique_amount)
    
    print("Run for video '" + str(video_id) + "' took " + str(round(time.time()-model_start,2)) + " seconds.")

In [ ]:
# Create an empty dataframe for storing all time dfs later
total_time = pd.DataFrame()

loop_start = time.time()
# Run all models on all videos
for video in video_ids:
    
    # Run all models on one video
    loop(video)
    
    # Extract all model times from video and save to time_df
    time_df = pd.DataFrame({"Video ID":time_VID,"IC":time_IC,"IS":time_IS,"OD":time_OD,"MC":time_MC, \\
              "SR":time_SR,"NR":time_NR,"TR":time_TR,"Frames":time_amount,"Unique Frames":unique_amount})
    
    # Concatenate total_time df and time_df of new video
    total_time = pd.concat([total_time,time_df])
    
    # Save Runtime dataframe as csv
    time_df.to_csv("Runtime.csv",index=False)
    
print("Loop through all videos took " + str(round(time.time()-loop_start,2)) + " seconds.")

# Load Summarizations
results = pd.read_csv("Summarizations.csv")

# Evaluate all predictions
evaluation = Pipeline.evaluate_predictions(results)

# Save evaluation dataframe as csv
evaluation.to_csv("Evaluation.csv",index=False)